# Homework 3: Vector Search

In this homework, we'll experiemnt with vector with and without Elasticsearch

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Getting the embeddings model

First, we will get the embeddings model multi-qa-distilbert-cos-v1 from the Sentence Transformer library

```python
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27

In [1]:
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

In [2]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)
user_question = 'I just discovered the course. Can I still join it?'

embeddings = embedding_model.encode(user_question)
print(f'First value in embedding: {embeddings[0]}')

First value in embedding: 0.0782226175069809


## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

```
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
```

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents.

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

* Create a list `embeddings`
* Iterate over each document
* `qa_text = f'{question} {text}'`
* compute the embedding for `qa_text`, append to `embeddings`
* At the end, let `X = np.array(embeddings)` (`import numpy as np`)

What's the shape of X? (`X.shape`). Include the parantheses.

In [3]:
import pandas as pd
import requests

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
pd.DataFrame(documents).head()

,text,section,question,course,id
0,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,data-engineering-zoomcamp,c02e79ef
1,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,data-engineering-zoomcamp,1f6520ca
2,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,data-engineering-zoomcamp,7842b56a
3,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,data-engineering-zoomcamp,0bbf41ec
4,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,data-engineering-zoomcamp,63394d91


In [4]:
mlzc_docs = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
print(len(mlzc_docs))

375


In [5]:
import numpy as np

embeddings = [embedding_model.encode(f"{doc['question']} {doc['text']}") for doc in tqdm(mlzc_docs)]
X = np.array(embeddings)

  0%|          | 0/375 [00:00<?, ?it/s]

In [6]:
X.shape

(375, 768)

## Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:

```python
scores = X.dot(v)
```

What's the highest score in the results?

* 65.0
* 6.5
* 0.65
* 0.065

In [7]:
v = embedding_model.encode(user_question)
max(X.dot(v))

0.6506574

## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]
```

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

If you don't understand how the search function work:


* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine here

(Note: you can replace argsort with argpartition to make it a lot faster)

In [8]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

In [9]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [10]:
df_ground_truth.head()

,question,course,document
2123,Where can I sign up for the course?,machine-learning-zoomcamp,0227b872
2124,Can you provide a link to sign up?,machine-learning-zoomcamp,0227b872
2125,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0227b872
2126,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0227b872
2127,How can I structure my questions and answers f...,machine-learning-zoomcamp,0227b872


In [11]:
search_engine = VectorSearchEngine(documents=mlzc_docs, embeddings=X)

matches = []

for m in tqdm(ground_truth):
    results = search_engine.search(embedding_model.encode(m['question']), num_results=5)
    matches.append(m['document'] in [d['id'] for d in results])

hit_rate = sum(matches) / len(ground_truth)
print(f'hit rate: {hit_rate:.3f}')

  0%|          | 0/1830 [00:00<?, ?it/s]

hit rate: 0.940


## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [12]:
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
index_settings = dict(
    settings=dict(number_of_shards=1, number_of_replicas=0),
    mappings=dict(
        properties=dict(
            id=dict(type='text'),
            text=dict(type='text'),
            section=dict(type='text'),
            question=dict(type='text'),
            course=dict(type='keyword'),
            question_vector=dict(type='dense_vector', index=True, similarity='cosine', dims=X.shape[1]),
        )
    )
)
index_name = 'course-questions'

es.indices.delete(index=index_name, ignore_unavailable=True)   # clear previously loaded, if any
es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [13]:
for doc, embedding in tqdm(zip(mlzc_docs, X), desc='Indexing', total=len(mlzc_docs), unit='document'):
    es.index(index=index_name, document=doc | dict(question_vector=embedding))

Indexing:   0%|          | 0/375 [00:00<?, ?document/s]

In [14]:
knn = dict(
    field='question_vector',
    query_vector=v,
    k=5,
    num_candidates=5,
    filter=dict(term=dict(course='machine-learning-zoomcamp')),
)
query_template = dict(knn=knn, _source=['id'])
response = es.search(index=index_name, body=query_template)
sorted_response = sorted(response['hits']['hits'], key=lambda m: m['_score'], reverse=True)

print(f'ID of doc with highest score:', sorted_response[0]['_source']['id'])

ID of doc with highest score: ee58a693


## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33
